# Combined Data Processing for _Illusory Tempo - Motor_ and _Illusory Tempo - Motor Split Range_

In [ ]:
import re
import json
import numpy as np
import pandas as pd
import scipy as sp
from glob import glob

AUDIOMETRY_LEVELS = ['125', '250', '500', '1000d', '1000u', '2000', '4000', '8000']

## Load all data from both experiments into a single table

In [ ]:
# Read audiometry data
audiometry_data1 = pd.read_csv('../E1/data/audiometry.csv')
audiometry_data2 = pd.read_csv('../E2/data/audiometry.csv')

# Find all raw session data files by searching in the data folder
datafiles1 = glob('../E1/data/ITM_*.csv')
datafiles2 = glob('../E2/data/ITMSR_*.csv')

# Load each data file and concatenate them into a single table per experiment
d1 = pd.concat((pd.read_csv(f) for f in datafiles1))
d2 = pd.concat((pd.read_csv(f) for f in datafiles2))

# Drop pilot participants from the start of Experiment 1
d1 = d1[d1.version >= 1.1]

# Copy spontaneous motor tempo data to its own table
smt_data1 = d1[d1.event == 'SPR'].reset_index()
smt_data2 = d2[d2.event == 'SPR'].reset_index()

# Select trial response events
d1 = d1[d1.event == 'trial'].reset_index()
d2 = d2[d2.event == 'trial'].reset_index()

# Convert register and pitch condition values from each experiment into MIDI note numbers
d1['pitch'] = d1['octave'] * 12 + 21
is_upper = d2.register == 'U'
d2['octave'] = d2.pitch / 2 + (~is_upper * 2) + (is_upper * 4.5)
d2['pitch'] = d2['octave'] * 12 + 21

# Set register of all Experiment 1 participants to 'B' ("both"). This makes filtering a lot easier later.
d1['register'] = 'B'

# Concatenate data
data = pd.concat((d1, d2), axis=0, join="outer", ignore_index=True)
smt_data = pd.concat((smt_data1, smt_data2), axis=0, join="outer", ignore_index=True)
audiometry_data = pd.concat((audiometry_data1, audiometry_data2), axis=0, join="outer", ignore_index=True)

### Process spontaneous motor tempo (SMT) task

Note that the SMT task is labeled SPR (spontaneous production rate) in the data file and in some code comments. However, SMT is a more appropriate term for what our task measured, and we adjusted our terminology partway through the project. SPR usually refers to the preferred tempo of motor actions used for producing sound, whereas SMT refers to the preferred tempo of motor actions not for the sake of sound production.

In [ ]:
# SMTs will be stored in a dictionary with one entry per participant
smt_dict = dict()

# Each row of the smt_data dataframe represents a different participant
for i, trial in smt_data.iterrows():

    # Convert tap times from a string to a list of timestamps
    tap_times = json.loads(trial.tap_times)

    # Convert tap times to intertap intervals (ITIs) by taking their difference
    itis = np.diff(tap_times)

    # Convert ITIs to their z-scores to detect any extreme outliers
    z_itis = sp.stats.zscore(itis)
    outlier = np.abs(z_itis) > 3

    # Calculate the SMT as the mean of non-outlier ITIs
    smt_dict[trial.subject] = np.mean(itis[~outlier])

### Process synchronization-continuation task

During this process, we need to remove false double-taps resulting from the FSR mistakenly detecting a release while still pressed. We can detect a false double-tap by the fact that its release time will be extremely close to the tap time and the subsequent tap will be detected the minimum permitted ms after the false release. Double-taps should be revised such that the real tap's release time and the false tap's tap time are both eliminated - the false tap's release is actually the first tap's release time.

#### DEMO: Final tap is fake

tap_times =        np.array(\[[     0, 500, 1000, 1500, 1601]\])
release_times =    np.array(\[[   100, 600, 1100, 1521, 1700]\])

down_times =       np.array(\[[   100, 100,  100,   21,   99]\])
bounce_times =     np.array(\[[np.inf, 400,  400,  400,   80]\])

is_false_release = np.array(\[[False, False, False,  True, False]\])
is_false_tap =     np.array(\[[False, False, False, False,  True]\])

good_taps =     np.array(\[[     0, 500, 1000, 1500]\])
good_releases = np.array(\[[   100, 600, 1100, 1700]\])

In [ ]:
# Set the trial parameters
nsync_tones = 8
ncont_tones = 16
ntones = nsync_tones + ncont_tones
max_taps = ntones * 3
ntrials = len(data)

# Create matrices that will hold trial data
tone_times = np.full((ntrials, ntones), np.nan)
tap_times = np.full((ntrials, max_taps), np.nan)
release_times = np.full((ntrials, max_taps), np.nan)
peak_pressures = np.full((ntrials, max_taps), np.nan)
peak_times = np.full((ntrials, max_taps), np.nan)
tap_types = np.full((ntrials, max_taps), '')

# Process each trial independently
num_false = 0
num_true = 0
for i, trial in data.iterrows():

    ###
    # Parse data
    ###

    # Convert tap and tone data from strings to lists
    trial_tones = np.array(json.loads(trial.tone_times), dtype=float)
    trial_taps = np.array(json.loads(trial.tap_times), dtype=float)
    trial_releases = np.array(json.loads(trial.release_times), dtype=float)
    trial_pressures = np.array(json.loads(trial.peak_pressures), dtype=float) if trial.experiment == 'ITMSR' else np.full_like(trial_releases, np.nan)
    trial_peaktimes = np.array(json.loads(trial.peak_times), dtype=float) if trial.experiment == 'ITMSR' else np.full_like(trial_releases, np.nan)
    trial_taptypes = np.array([s[1] for s in trial.tap_types[1:-1].split(", ")])

    # If the final release occurred after the trial ended, add a nan to the end of trial_releases, pressures, and peaktimes
    if len(trial_releases) == len(trial_taps) - 1:
        trial_releases = np.append(trial_releases, np.nan)
        trial_pressures = np.append(trial_pressures, np.nan)
        trial_peaktimes = np.append(trial_peaktimes, np.nan)

    ###
    # Remove presumed fake taps
    ###

    # Set the minimum time from release to tap (bounce) and from tap to release (down) allowed
    # These should be set based on the settings in the Arduino routine
    min_bounce = 80
    min_down = 20
    tolerance = 10  # The distribution of tap releases that precede 80 ms bounce times suggests that false releases typically occur within 30 ms of the tap onset

    # is_false_release indicates the ith tap may have false-released.
    # This occurs when the release time is suspiciously close to the minimum allowed.
    # The can happen if the tap pressure oscillates above and below threshold as the person's finger lands.
    down_times = trial_releases - trial_taps
    is_false_release = down_times <= min_down + tolerance

    # is_false_tap indicates the ith tap may be false.
    # This occurs when the time since the last release is suspiciously close to the minimum allowed.
    # This can happen if a false release occurred and the person's finger stays down for longer than the
    # Arduino's debounce period.
    bounce_times = np.full(len(trial_taps), np.nan)
    bounce_times[0] = np.inf
    bounce_times[1:] = trial_taps[1:] - trial_releases[:-1]
    is_false_tap = bounce_times == min_bounce

    # Only treat as bad if the ith release and (i+1)th tap are both potentially false.
    # This suggests a false release occurred and the person's finger remained down longer than the debounce.
    is_bad = is_false_release[:-1] & is_false_tap[1:]
    num_false += np.sum(is_bad)
    num_true += np.sum(~is_bad)
    is_false_release[:-1] = is_bad
    is_false_tap[1:] = is_bad

    # Get just the good tap times
    good_taps = trial_taps[~is_false_tap]

    # Get just the good release times
    good_releases = trial_releases[~is_false_release]

    # Remove other data associated with fake taps
    good_pressures = trial_pressures[~is_false_tap]
    good_peaktimes = trial_peaktimes[~is_false_tap]
    good_types = trial_taptypes[~is_false_tap]

    ###
    # Finalization
    ###

    tone_times[i, :] = trial_tones
    tap_times[i, :len(good_taps)] = good_taps
    release_times[i, :len(good_releases)] = good_releases
    peak_pressures[i, :len(good_pressures)] = good_pressures
    peak_times[i, :len(good_peaktimes)] = good_peaktimes
    tap_types[i, :len(good_types)] = good_types
    
print('False tap rate:', num_false / (num_true+num_false))

We next proceed to the audiometry processing, the calculation of synchronization tap phases relative to the preceding stimulus, and the inter-tap intervals during continuation tapping. When a tap is processed, it is also automatically marked as usable or not, based on the exclusion criteria described in the Data Analysis section of our manuscript. Note that conversion of tapping phases to asynchronies is performed separately in the Combined Analysis.ipynb file.

In [ ]:
total_taps = np.sum(np.isin(tap_types, ['S', 'C']))
long_data = pd.DataFrame(None, index=range(total_taps), columns=[
    'subject', 'experiment', 'version', 'experimenter', 'handedness', 'smt', 'block', 'trial',  # Trial metadata
    'register', 'pitch', 'ioi', 'hearing_threshold',  # Independent variables
    'tap_type', 'interval_number', 'tap_duration', 'peak_pressure',  # Tap metadata
    'tap_phase', 'tap_peak_phase',  # Synchronization phase scores
    'iti', 'peak_pressure_iti', 'rel_iti',  # Continuation phase scores
    'usable'  # Label each trial/tap as bad or usable
])

df_idx = 0
prev_subj = None
spline = None
for i, trial in data.iterrows():

    # Infer the block number from the trial number. Note that Experiment 1 had an error in which
    # blocks 1-4 were 12 trials each, and block 5 was 72 trials (instead of 24 trials for each)
    trial_number = 1 if trial.subject != prev_subj else trial_number + 1
    block_number = ((trial_number - 1) // 24) + 1 if trial.experiment == 'ITMSR' else min(5, ((trial_number - 1) // 12) + 1)

    # Identify the time of the first continuation tone. This time can be used to separate sync taps and cont taps.
    first_cont_tone_time = tone_times[i, nsync_tones]

    ###
    # AUDIOMETRY PROCESSING
    ###

    # If this is a participant's first trial, process their audiogram
    if trial.subject != prev_subj:

        # Load participant's audiogram
        subj_audiometry = audiometry_data.loc[audiometry_data.subject == trial.subject]
        freqs = []
        thresholds = []
        reps = 1
        for freq in AUDIOMETRY_LEVELS:

            thresh = subj_audiometry[freq].iloc[0]
            if not np.isnan(thresh):

                # Convert strings in audiogram table column headers to floats
                num_freq = float(re.sub(r'\D', '', freq))

                # If the same frequency was sampled multiple times, as for 1000 Hz in IT-M, average the thresholds from all repetitions
                if len(freqs) > 0 and num_freq == freqs[-1]:
                    reps += 1
                    thresholds[-1] = thresholds[-1] * (reps - 1) / reps + thresh * 1 / reps

                # Otherwise, just add the frequency and threshold to our data arrays for use in fitting the cubic spline
                else:
                    reps = 1
                    freqs.append(num_freq)
                    thresholds.append(thresh)

        # Fit cubic spline that we can use to interpolate/extrapolate threshold values for all stimuli
        # Because frequencies are perceived log-linearly, we interpolate thresholds on the log of frequency
        log_freqs = np.log2(freqs)
        spline = sp.interpolate.CubicSpline(log_freqs, thresholds, bc_type='natural', extrapolate=True)

    # Convert the MIDI number for the current trial's pitch to its corresponding frequency in Hz
    trial_freq = 440 * 2 ** ((trial.pitch - 69) / 12)

    # Use the participant's audiogram to interpolate their hearing threshold for the current trial's stimulus
    hearing_threshold = spline(np.log2(trial_freq))

    ###
    # TAP PROCESSING
    ###

    for j, tap_time in enumerate(tap_times[i, :]):

        # Skip over padding in tap time matrix
        if np.isnan(tap_time):
            continue

        # Reset data values for the current tap
        interval_num = np.nan
        tap_phase = np.nan
        peak_phase = np.nan
        iti = np.nan
        ipi = np.nan
        rel_iti = np.nan
        usable = np.nan

        # Calculate the tap's offset relative to all tones
        offsets = tap_time - tone_times[i, :]

        # Determine whether the tone occurred during synchronization or continuation
        tap_type = tap_types[i, j]

        # Skip taps that occurred before the first tone
        if tap_type == 'B':
            continue

        # SYNCHRONIZATION TAP HANDLING
        # NOTE: The tap that triggers the first continuation tone is analyzed as a sync tap.
        elif tap_type == 'S' or tap_time <= first_cont_tone_time:

            # Mark the first continuation tap as both synchronization and continuation
            if tap_type == 'C':
                tap_type = 'SC'

            # Find all preceding tones, i.e., any that the tap has a non-negative offset from
            all_preceding_tones = np.where(offsets >= 0)[0]

            # Find the latest preceding tone and consider it the start of interval during which the tap occurred
            preceding_tone_num = np.max(all_preceding_tones)
            interval_num = preceding_tone_num + 1
            interval_start = tone_times[i, preceding_tone_num]

            # Determine what fraction of the way through the interval the tap occurred, then convert to phase in radians
            tap_phase = (tap_time - interval_start) / trial.ioi
            tap_phase *= 2 * np.pi

            # Do the same for the peak pressure
            peak_phase = (peak_times[i, j] - interval_start) / trial.ioi
            peak_phase *= 2 * np.pi

            # Sync tap exclusion:
            # 1) An estimated 80+% of 0-asynchrony taps are due to the Arduino not detecting taps while initializing
            # the tone, so exclude any sync tap with an asynchrony of 0
            # 2) The tap that triggered the first continuation tone should only be analyzed if it occurred before the
            # hypothetical 9th sync tone would have occurred; otherwise phase may be affected by the tone not playing
            usable = (tap_phase > 0) & (tap_phase < 2 * np.pi)

        # CONTINUATION TAP HANDLING
        # NOTE: The tap that triggers the first continuation tone is analyzed as a sync tap.
        elif tap_type == 'C' and not tap_time <= first_cont_tone_time:

            # Find the interval number that this tap ended, based on the index of the last preceding tone
            all_preceding_tones = np.where(offsets > 0)[0]
            preceding_tone_num = np.max(all_preceding_tones)
            interval_num = preceding_tone_num + 1

            # Calculate the intertap interval and inter-peak-pressure interval preceding the current tap
            iti = tap_times[i, j] - tap_times[i, j-1]
            ipi = peak_times[i, j] - peak_times[i, j-1]

            # Compare the ITI to the target ITI (the IOI of that trial)
            rel_iti = iti / trial.ioi

            # Continuation tone exclusion:
            # 1) An ITI greater than twice the stimulus IOI indicates a missed tap or that the person paused
            # 2) An ITI less than half the stimulus IOI indicates the person accidentally tapped early or was trying to speed through the trial
            usable = .5 <= rel_iti <= 2

        # Sanity check
        else:
            raise ValueError('Invalid tap type detected: %s' % tap_type)

        # Determine the duration and max pressure of the tap
        # NOTE: If the last tap was released after the trial ended, both values will come out as NaN
        tap_duration = release_times[i, j] - tap_time
        tap_pressure = peak_pressures[i, j]

        # Input scores into the full table
        long_data.loc[df_idx] = dict(

            # Trial metadata
            subject=trial.subject,
            experiment=trial.experiment,
            version=trial.version,
            experimenter=trial.experimenter,
            handedness=trial.handedness,
            smt=smt_dict[trial.subject],
            block=block_number,
            trial=trial_number,

            # Independent variables
            register=trial.register,
            pitch=trial.pitch,
            ioi=trial.ioi,
            hearing_threshold=hearing_threshold,

            # Tap metadata
            tap_type=tap_type,
            interval_number=interval_num,
            tap_duration=tap_duration,
            peak_pressure=tap_pressure,

            # Synchronization phase scores
            tap_phase=tap_phase,
            tap_peak_phase=peak_phase,

            # Continuation phase scores
            iti=iti,
            rel_iti=rel_iti,
            peak_pressure_iti=ipi,

            # Usability of data (1 if good, 0 if bad)
            usable=int(usable)
        )

        df_idx += 1
        prev_subj = trial.subject

## Save data
At this stage we have a (very) long table in which each row contains the data for one tap, and the table contains data for all taps recorded in both experiments.

In [ ]:
long_data.to_csv('../processed_data/processed_taps.csv', index=False)